# 🏦 Enterprise Credit Approval ML Pipeline (V3.5)

--- 

**Clean Architecture + Professional MLOps Hybrid Framework**

Bu notebook, **modüler Python paketlerini (`src/`)** kullanarak uçtan uca bir kredi onay sistemi çalıştırır. Proje, sadece teknik metrikleri (AUC/F1) değil, kurumsal kararlar için kritik olan **ROI**, **NPV** ve **Amortisman** gibi finansal değerleri de hesaplar.

---

## 1️⃣ [CELL 1] Environment & Infrastructure Setup

Bu hücrede Google Colab ortamı doğrulanır, GPU tespiti yapılır ve gerekli kütüphaneler yüklenir.

In [ ]:
import sys
import os
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

print("🔄 Sistem kontrol ediliyor...")

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab Ortamı Tespit Edildi.")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    
    # PROJE YOLUNU BURAYA GİRİN (Drive üzerindeki klasör adı)
    PROJECT_PATH = '/content/drive/MyDrive/credit-approval'
    
    if os.path.exists(PROJECT_PATH):
        os.chdir(PROJECT_PATH)
        if PROJECT_PATH not in sys.path:
            sys.path.append(PROJECT_PATH)
        print(f"✅ Çalışma dizini: {PROJECT_PATH}")
    else:
        print(f"⚠️ HATA: {PROJECT_PATH} bulunamadı! Lütfen Drive'a yüklediğinizden emin olun.")
else:
    print("💻 Yerel (Local) Ortam Tespit Edildi.")
    PROJECT_PATH = os.getcwd()
    print(f"✅ Çalışma dizini: {PROJECT_PATH}")

In [ ]:
# Bağımlılıkları Yükle
if IN_COLAB:
    print("📦 Kütüphaneler yükleniyor (requirements.txt)...")
    !pip install -r requirements.txt
else:
    print("ℹ️ Yerel çalışma: pip install atlandı.")

## 2️⃣ [CELL 2-7] Enterprise ML Pipeline Execution

Pipeline şu aşamalardan geçer:
1. **Veri Yükleme & Doğrulama** (Cell 2)
2. **Öznitelik Mühendisliği** (Cell 3)
3. **Model Eğitimi & Optimizasyon** (Cell 4)
4. **İstatistiksel Doğrulama (Friedman Test)** (Cell 5)
5. **Akıllı Model Seçimi & Validasyon** (Cell 6)
6. **İş Etkisi Analizi (ROI/NPV)** (Cell 7)

In [ ]:
from src.core.config import get_config
from src.core.logger import setup_logger
from src.pipelines.training_pipeline import TrainingPipeline

# 1. Konfigürasyonu Yükle
config = get_config(reload=True)
config.optuna_trials = 30  # Hızlı demo için trial sayısı düşürülmüştür

# 2. Logger Hazırla
logger = setup_logger().logger

# 3. Pipeline'ı Başlat
pipeline = TrainingPipeline(config=config, logger=logger)

try:
    results = pipeline.run()
    print(f"\n🏆 PIPELINE TAMAMLANDI! Seçilen Model: {results['best_model']}")
except Exception as e:
    print(f"❌ Çalıştırma hatası: {e}")

## 3️⃣ [RESULTS] Advanced Dashboards & Stakeholder Reports

Aşağıda, pipeline tarafından otomatik üretilen **kurumsal analiz panelleri** ve raporlar sunulmaktadır.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

def display_dashboard(title, filename):
    path = Path(config.output_dir) / config.plots_dir / filename
    if path.exists():
        plt.figure(figsize=(15, 12))
        img = mpimg.imread(str(path))
        plt.imshow(img)
        plt.axis('off')
        plt.title(title, fontsize=14, fontweight='bold')
        plt.show()
    else:
        print(f"⚠️ Görsel bulunamadı: {filename}")

# 1. Model Eğitim Özeti
display_dashboard("1. Training Results Dashboard", "training_results_dashboard.png")

# 2. Model Seçim ve Hazırlık Analizi
display_dashboard("2. Advanced Model Selection Dashboard", "model_selection_dashboard.png")

# 3. Kurumsal İş Etkisi (Enterprise Dashboard)
display_dashboard("3. 12-Panel Business Impact Analysis", "business_impact_extended.png")

### 📜 Executive Summary & Business Case

Otomatik oluşturulan metin tabanlı raporlar aşağıdadır:

In [ ]:
def print_report(filename):
    path = Path(config.output_dir) / config.results_dir / filename
    if path.exists():
        print(f"\n--- [ {filename} ] ---\n")
        with open(path, 'r', encoding='utf-8') as f:
            print(f.read())
    else:
        print(f"⚠️ Rapor bulunamadı: {filename}")

print_report("business_case.txt")

## 🧹 Memory Optimization

Belleği temizlemek için aşağıdaki hücreyi çalıştırabilirsiniz.

In [ ]:
import gc
import torch

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("🧹 Memory and GPU cache cleared.")